<a href="https://colab.research.google.com/github/componavt/differential_equations/blob/main/src/hill_equation/93_gamma_correct_y_without_initial_shifts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧬📊 Solution Archive for Gene Regulatory ODE System

**Overview:**  
This notebook computes and saves numerical solutions of a two-dimensional gene-regulatory ODE system under a wide range of parameter values. Solutions are saved to disk for later visualization in interactive apps like Streamlit.

---

**Model Equations**  
\begin{cases}
\frac{dx}{dt} = \frac{K\,x^{1/α}}{b^{1/α} + x^{1/α}} \;-\; γ_1\,x,\\[6pt]
\frac{dy}{dt} = \frac{K\,y^{1/α}}{b^{1/α} + y^{1/α}} \;-\; γ_2\,y.
\end{cases}

**Fixed parameters:**  
- b = 1.0, K = 1.0  
- Time: t ∈ [0, 1.0], N = 500 steps  
- Initial condition(s): x₀ = y₀ = b * (1 - ε), ε ≈ 1e-10

**Parameter sweeps:**  
- α ∈ {1e-9, 1e-10, … , 1e-14}  
- γ₁, γ₂ ∈ {0.0, 0.5, 1.0}  
- Solvers: RK45, DOP853, BDF

---

**What this notebook does:**  
1. For each combination of α, γ₁, γ₂, and solver:
   - Integrates the ODE system from a fixed initial condition.
   - Saves time points and trajectories x(t), y(t).
2. Each solution is stored as a dictionary with:
   - Parameters: α, γ₁, γ₂, solver name, initial values
   - Solution arrays: t, x(t), y(t)
   - Status: success or failure
3. All results are saved in a single `pickle` file:  
   📁 `ode_solutions/solutions.pkl`

---

**How to use this in Streamlit:**  
- Load the file with `pickle.load(open(...))`  
- Add controls (checkboxes, sliders) for α, γ₁, γ₂, solver  
- Filter solutions and display interactive plots of x(t), y(t)  

---

*This notebook provides a scalable, solver-agnostic framework for collecting ODE simulation results for downstream visualization and analysis.*

In [13]:
# Modified code for saving ODE solutions for Streamlit
# Cell 1: Parameters and library imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pickle
import time
from matplotlib import cm
from numpy.linalg import eigvals
import os

b = 1.0
#t_end = 0.05
t_end = 1
K = 1.0
# alpha_list = [1e-4 * (10 ** -i) for i in range(10)]  # 1e-4 to 1e-13
alpha_list = [1e-9 * (10 ** -i) for i in range(6)]  # 1e-9 to 1e-14
#alpha_list = [1e-9 * (10 ** -i) for i in range(9)]  # 1e-9 to 1e-17
print("Alpha values:", alpha_list)

N = 500
t_eval = np.linspace(0, t_end, N)

MAX_LOG_MESSAGES = 3
#all_methods = ["RK45", "DOP853", "BDF", "Radau", "RK23", "Symplectic"]
all_methods = ["DOP853", "BDF", "Radau", "RK23",]  # limited for simplicity

error_counts = {m: 0 for m in all_methods}
error_counts.update({m + '_msgs': 0 for m in all_methods})
alpha_failed = {m: [] for m in all_methods}


# positive gamma only
gamma_1_list = np.arange(0, 1.1, 0.5)
gamma_2_list = np.arange(0, 1.1, 0.5)
initial_conditions = [(b * (1 - 1e-10), b * (1 - 1e-10))]
style_map = {}

# Folder for saving data
output_folder = "ode_solutions"
os.makedirs(output_folder, exist_ok=True)

Alpha values: [1e-09, 1.0000000000000002e-10, 1.0000000000000001e-11, 1.0000000000000002e-12, 1.0000000000000002e-13, 1.0000000000000002e-14]


In [14]:
# Cell 2: Define RHS with decay terms
def get_rhs(alpha, gamma1, gamma2):
    def rhs(t, xy):
        x = np.clip(xy[0], 1e-20, 1e20)
        y = np.clip(xy[1], 1e-20, 1e20)
        inv_alpha = 1 / alpha
        x_alpha = np.exp(np.clip(np.log(x) * inv_alpha, -700, 700))
        y_alpha = np.exp(np.clip(np.log(y) * inv_alpha, -700, 700))
        b_alpha = np.exp(np.clip(np.log(b) * inv_alpha, -700, 700))
        dxdt = K * x_alpha / (b_alpha + x_alpha) - gamma1 * x
        dydt = K * y_alpha / (b_alpha + y_alpha) - gamma2 * y
        return [dxdt, dydt]
    return rhs

In [15]:
# Cell 3: Solve and save all results
data = []

for alpha in alpha_list:
    for gamma1 in gamma_1_list:
        for gamma2 in gamma_2_list:
            for method in all_methods:
                for x0, y0 in initial_conditions:
                    rhs = get_rhs(alpha, gamma1, gamma2)
                    try:
                        sol = solve_ivp(rhs, [0, t_end], [x0, y0], t_eval=t_eval, method=method)
                        entry = {
                            'alpha': alpha,
                            'gamma1': gamma1,
                            'gamma2': gamma2,
                            'method': method,
                            'x0': x0,
                            'y0': y0,
                            't': sol.t,
                            'x': sol.y[0],
                            'y': sol.y[1],
                            'success': sol.success
                        }
                        data.append(entry)
                    except Exception as e:
                        print(f"Error with alpha={alpha}, gamma1={gamma1}, gamma2={gamma2}, method={method}: {e}")

# Cell 4: Save to pickle file
with open(os.path.join(output_folder, "solutions.pkl"), "wb") as f:
    pickle.dump(data, f)

print(f"Saved {len(data)} solution entries to 'solutions.pkl'")

Saved 216 solution entries to 'solutions.pkl'


# 🧬📉 Анализ фазового пространства генрегуляторной системы — обновлённая версия

- **Цель**: Изучить фазовую динамику, бифуркации и чувствительность решений системы ОДУ, моделирующей генрегуляцию:  
  $$
  \frac{dx}{dt} = \frac{K \cdot x^{1/\alpha}}{b^{1/\alpha} + x^{1/\alpha}}, \quad
  \frac{dy}{dt} = \frac{K \cdot y^{1/\alpha}}{b^{1/\alpha} + x^{1/\alpha}}
  $$

- **Фиксированные параметры**:  
  - $b = 1.0$  
  - $K = 1.0$ (фиксированный скалярный множитель)  

- **Начальные условия**: Несколько точек около $b$:  
  $$
  x_0, y_0 \approx b \times (1 - \epsilon), \quad \epsilon \in \{10^{-7}, 10^{-8}, 10^{-9}\}
  $$  
  для исследования чувствительности и возможности нескольких траекторий решения.

- **Временной интервал**:  
  $$
  t \in [0, t_{\text{end}}], \quad t_{\text{end}} = 0.05
  $$

- **Диапазон альфа** (логарифмический масштаб, суженный):  
  $$
  \alpha \in \{10^{-9}, 10^{-10}, 10^{-11}, 10^{-12}, 10^{-13}, 10^{-14}\}
  $$

## ⚙️ Используемые численные решатели
- ⚡ `RK45` — явный метод Рунге-Кутты 4(5) с адаптивным шагом  
- 🌐 `DOP853` — высокопорядковый метод Dormand-Prince 8(5,3)  
- 🧱 `BDF` — метод обратных разностей, для жёстких систем  
- 🧿 `Radau` — A-устойчивый неявный метод Рунге-Кутты  
- 🐢 `RK23` — явный метод Рунге-Кутты 2(3), пониже порядок  
- 🌀 `Symplectic` — собственная реализация явного метода Эйлера (симплектический аналог)

## 📈 Визуализация и диагностика
- ✔️ Фазовые портреты ($y(t)$ против $x(t)$) для каждого решателя и каждого $\alpha$, на каждом графике показаны траектории для разных начальных условий.  
- ➡️ Стрелки (quiver) вдоль траекторий показывают направление потока — размер стрелок подобран для наглядности без засорения.  
- 🔄 Цвета, стили линий, маркеры используются для различения начальных условий; значения начальных условий подписаны прямо на кривых.  
- ⚠️ Учёт и логирование численных ошибок и сбоев для каждого решателя и $\alpha$.  
- 📊 Итоговая таблица по решателям с:  
  - Общим временем интегрирования  
  - Числом вычислений функции правой части (`nfev`), отражающим вычислительные затраты.

## 🧪 Расширенный численный анализ
- 🔍 **Оценка показателя Ляпунова**:  
  Приблизительный расчёт на основе расходимости близких траекторий.  
  Положительный показатель говорит о чувствительной зависимости и потенциальном хаосе.  
- 🌪️ **Обнаружение бифуркации Неймарк–Сакера**:  
  - Оценка отображения Пуанкаре (конечная точка решения)  
  - Якобиан, вычисленный численно по разностям начальных условий  
  - Анализ собственных чисел для выхода за единичную окружность (бифуркация).

## 📌 Примечания по реализации
- Начальные условия варьируются вблизи $b$ для проверки множественности решений и чувствительности.  
- Управление переполнением реализовано через ограничение экспонент в логарифмической области для предотвращения `NaN`/`Inf`.  
- Симплектический решатель — упрощённый явный метод Эйлера, поскольку в SciPy нет нативного симплектического метода.  
- Ошибки и неудачи интегрирования собираются и визуализируются в зависимости от $\alpha$, что помогает локализовать проблемные зоны параметра.

---

*Данный ноутбук даёт основу для исследования сложной динамики генрегуляторных ОДУ-систем, включая численную устойчивость, бифуркационный анализ и выявление хаоса с помощью множества решателей и перебора параметров.*